# **Mini Project 5: 대출 위험도 평가** 

# 1. EDA

## Data load & Analysis

In [ ]:
# for colab
# cd '/content/drive/MyDrive'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('cs_data.csv', index_col=0)
data.head()

In [ ]:
data.describe()

In [ ]:
sns.countplot(x='SeriousDlqin2yrs', data=data, palette='RdBu_r')
plt.title('Binary Class Comparison')

In [ ]:
data['NumberOfTime60-89DaysPastDueNotWorse'].value_counts()

In [ ]:
data['NumberOfTimes90DaysLate'].value_counts()

In [ ]:
data['NumberOfTime30-59DaysPastDueNotWorse'].value_counts()

In [ ]:
data['NumberOfDependents'].value_counts()

### **Analysis**
- 먼저 data를 load하면서 전체적인 요소 및 값들의 분포를 파악했다.

- 첫 번째로 target data (SeriousDlqin2yrs)에 대한 불균형도가 심한 것을 확인 할 수 있다. 이는 추후에 불균형된 학습을 유발 할 수 있으므로, 이를 해소하기 위한 방법으로 imblearn에서 SMOTE 모듈을 이용해 oversampling 통해서 어느 정도 해소해볼 예정이다.

- 두 번째로 돈을 연체하게 된 data를 살펴 보았는데, 96과 98이라는 이상치로 보이는 숫자가 있어 이를 값을 대체하거나 삭제할 예정이다.

## Data Visualization
- Data의 각종 요소별 시각화를 진행했다.

In [ ]:
# remove target variable Dlqin2yrs and variables with missing values
feature_list = list(data.columns.values)
remove_list = ['SeriousDlqin2yrs','MonthlyIncome','NumberOfDependents']
for each in remove_list:
    feature_list.remove(each)

for each in feature_list:
    sns.histplot(data[each], kde=True, bins=100)
    plt.show()

**Analysis**
- RevolvingUtilizationOfUnsecuredLines
- NumberOfTime30-59DaysPastDueNotWorse
- DebtRatio
- NumberOfTimes30DaysLate
- NumberRealEstateLoansOrLines
- NumberOfTime60-89DaysPastDueNotWorse

위의 6가지 속성에 대해서는 매우 편향된 분포를 가지고 있음을 파악할 수 있었다.

### DebtRatio (Boxplot)

In [ ]:
data['DebtRatio'].value_counts()

In [ ]:
data[data['DebtRatio'] > 1].value_counts()

In [ ]:
plt.boxplot(data['DebtRatio'])
plt.title('DebtRatio')
plt.ylim(-0.1, 2)
plt.show()

In [ ]:
data_sample = data.copy()
data_sample['age/20'] = data_sample[['age']].applymap(lambda x : int(x/20))
sns.boxplot(x='age/20', y='DebtRatio', data=data_sample)
plt.title('DebtRatio')
plt.ylim(-0.1, 3.0)
plt.show()

In [ ]:
data_sample[(data_sample['age/20']>=3) & (data_sample['DebtRatio']>1.)].describe()

In [ ]:
data.loc[data['age'] == 0, 'age'] = data['age'].median()

### **Analysis**
violin plot을 이용하여 연령별 DebtRatio를 파악해보았다. 그 결과, 60대 이상 (3번째 boxplot)에서 DebtRatio가 1을 넘어가는 값을 보였다. 이를 통해 data값이 잘못된 것일 지, 이 나이대에 특성이 그러한 지를 파악해 볼 필요가 있어 보였다.

그리고 추가적으로 나이가 '0'인 값에 대해서는 있을 수 없는 값으로 판단하여, 중간값으로 대체했다.

### RevolvingUtilizationOfUnsecuredLines (Boxplot)

In [ ]:
data['RevolvingUtilizationOfUnsecuredLines'].describe()

In [ ]:
data[data['RevolvingUtilizationOfUnsecuredLines'] > 1].value_counts()

In [ ]:
plt.boxplot(data['RevolvingUtilizationOfUnsecuredLines'])
plt.title('RevolvingUtilizationOfUnsecuredLines')
plt.ylim(-0.1, 1.5)
plt.show()

In [ ]:
data_sample = data.copy()
data_sample['age/20'] = data_sample[['age']].applymap(lambda x : int(x/20))
sns.boxplot(x='age/20', y='RevolvingUtilizationOfUnsecuredLines', data=data_sample)
plt.title('RevolvingUtilizationOfUnsecuredLines')
plt.ylim(-0.1, 1.1)
plt.show()

### **Analysis**
위와 마찬가지로 boxplot을 이용해 연령별로 현재 운용 가능한 현금의 비율을 살펴 보았다. 물론 비율이 1을 넘어가는 값은 이상치라고 생각되며, 적절한 값으로 대체를 하거나 삭제할 필요성은 있어 보인다. 연령은 높아질 수로 현재 운용가능한 현금의 비율이 낮아지는 것을 확인 할 수 있었고, 40~80대에서는 이상치로 판단 되는 값이 많은 것을 또한 확인했다.

### Age (Pie chart)

In [ ]:
age_df = pd.DataFrame(data.age)
age_df.head()

In [ ]:
# 20살 단위로 group화
age_df['age/20'] = age_df[['age']].applymap(lambda x : int(x/20))
age_df_count = age_df.groupby(['age/20'])['age/20'].count()
age_df_count.index = ['20<=age<40', '40<=age<60', '60<=age<80', '80<=age<100', '100<=age']
age_df_count.column = ['count']
age_df_count

In [ ]:
# data visualization
plt.figure(figsize=(7, 6))
plt.pie(age_df_count.values, labels=age_df_count.index, autopct='%1.1f%%')
plt.axis('equal')
plt.title('age count')
plt.show()

### **Analysis**
전체적인 나이 분포를 확인해본 결과, 40 ~ 60살 정도가 46.5% 이상의 높은 비율을 가지고 있었고 다음으로 60 ~ 80살이 28.3%로 40살 이상에서 약 75%가 넘는 분포를 가지고 있었다. 그 다음으로 20 ~ 40살 정도에서 약 21%의 비율을 차지했음을 알 수 있었다.

### SeriousDlqin2yrs (Violin Plot)

In [ ]:
df_SeriousDlqin2yrs = pd.DataFrame(data.SeriousDlqin2yrs)
df_SeriousDlqin2yrs.head()

In [ ]:
data['NumberOfTime60-89DaysPastDueNotWorse'].value_counts()

In [ ]:
plt.figure(figsize=(15, 10))
sns.violinplot('NumberOfTime60-89DaysPastDueNotWorse', 'age', hue='SeriousDlqin2yrs', data=data[data['NumberOfTime60-89DaysPastDueNotWorse'] < 8])
plt.title('SeriousDlqin2yrs')
plt.show()

### **Analysis**
violin plot을 이용하여 최근 2년간 60 ~ 89일 연체한 횟수에 따른 2년 동안 90일 이상 연체 여부를 파악해보려고 했다. 하지만 위의 그림과 같이 60 ~ 89일 연체한 횟수에 대해 이상치 (outlier)로 보이는 것이 탐지가 되었다.

이를 value_counts()를 통해서 다시 본 결과, 2년간 연체한 횟수가 98, 96번이라는 답변이 **264명과 5명**으로 조사되어 있다는 것을 파악했다. 이에 대한 값을 아예 **삭제를 할 것인지, 아니면 값을 대체할 지를** 결정이 필요해 보인다.

그리고 그림을 통해서 outlier 등을 제외한 나머지 분포를 살펴보았을 때에, 연체를 한 사람이 **가운데 연령대**에서 조금 더 높은 비율을 가진 것을 확인했습니다.

## 결측치 파악

In [ ]:
data.info()

In [ ]:
data.isna().sum()

### **Analysis**
결측치 (NA)에 대한 분석이다. 총 2개의 요소에서 결측치가 발생했다. (MonthlyIncome, NumberOfDependents) 특히, 월 수입에 대한 결측치는 거의 3만개에 가까운 숫자이기에 삭제하기에는 무리가 있다고 판단되었다. 그리고 부양 가족의 수도 그냥 삭제하기 보다는 데이터 전체의 평균 등으로 대체하여 사용할 예정이다.

구체적으로 위의 상관 관계(heat map)에서 볼 수 있듯이, MonthlyIncome은 NumberRealEstateLoansOrLines에서 그나마 높은 상관 관계를 가지기에 각 값 별로 가지고 있는 월 평균 수입을 MonthlyIncome 값으로 대체할 예정이다.

다음으로 NumberOfDependents의 경우, age와 음의 상관 관계이지만 관련성이 높아 보이므로 나이대에 따른 NumberOfDependents를 적용하거나 삭제할 예정이다.

## Data preprocessing

In [ ]:
preprocessing_data = data.copy()
preprocessing_data.head()

In [ ]:
partial_preprocessing_data = preprocessing_data[['MonthlyIncome','NumberOfDependents']]
#partial_preprocessing_data.dropna(how='any')
partial_preprocessing_data = partial_preprocessing_data.dropna(how='any')

sns.histplot(partial_preprocessing_data['MonthlyIncome'], kde=True, bins=100)
plt.show()
sns.histplot(partial_preprocessing_data['NumberOfDependents'], kde=True, bins=100)
plt.show()

### 결측치 처리 : NumberOfDependents

In [ ]:
preprocessing_data['age/10'] = preprocessing_data[['age']].applymap(lambda x : int(x/10))
preprocessing_data.head()

In [ ]:
mean_df = preprocessing_data.groupby('age/10')['NumberOfDependents'].mean()
mean_df = round(mean_df)
mean_df

In [ ]:
# NumberOfDependents의 결측치 채우기
for j in range(0, 11):
    for i in mean_df:
        preprocessing_data.loc[(preprocessing_data['NumberOfDependents'].isnull()) & (preprocessing_data['age/10'] == j),
                    'NumberOfDependents'] = round(i)

In [ ]:
preprocessing_data.info()

### 결측치 처리 : MonthlyIncome

In [ ]:
mean_df1 = preprocessing_data.loc[preprocessing_data['SeriousDlqin2yrs'] == 0, 'MonthlyIncome'].groupby(preprocessing_data['age/10']).mean()
mean_df1, mean_df1.shape

In [ ]:
mean_df2 = preprocessing_data.loc[preprocessing_data['SeriousDlqin2yrs'] == 1, 'MonthlyIncome'].groupby(preprocessing_data['age/10']).mean()
mean_df2, mean_df2.shape

In [ ]:
# MonthlyIncome 결측치 채우기

for i in mean_df1:
  for j in range(2, 11):
    preprocessing_data.loc[(preprocessing_data['SeriousDlqin2yrs'] == 0) & (preprocessing_data['MonthlyIncome'].isnull()) & (preprocessing_data['age/10'] == j),
              'MonthlyIncome'] = i

for i in mean_df2:
  for j in range(2, 11):
    preprocessing_data.loc[(preprocessing_data['SeriousDlqin2yrs'] == 1) & (preprocessing_data['MonthlyIncome'].isnull()) & (preprocessing_data['age/10'] == j),
              'MonthlyIncome'] = i

In [ ]:
preprocessing_data.drop('age/10', axis=1, inplace=True)
preprocessing_data.shape

In [ ]:
preprocessing_data.isna().any()

### Result
NumberOfDependents와 MonthlyIncome의 결측치를 위와 같이 다 채우는 방향으로 진행하였다.

구체적으로 NumberOfDependents의 경우에는 age를 10살 기준으로 나누어 각 평균값을 구하고, 각 나이가 해당되는 연령의 결측치에 평균값(int)으로 채웠다.

다음으로 MonthlyIncome는 SeriousDlqin2yrs와 연령별 기준에 대한 MonthlyIncome 평균값을 구했다. 이를 각 해당하는 대출 건수에 대한 결측치를 채웠다.

## Outlier (이상치)

In [ ]:
def get_outlier(df=None, column=None, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  
  lowest = quantile_25 - IQR*weight
  highest = quantile_75 + IQR*weight
  lowest_outlier_idx = df[column][df[column] < lowest].index
  highest_outlier_idx = df[column][df[column] > highest].index
  return lowest, highest, lowest_outlier_idx, highest_outlier_idx

### 이상치 제거 : RevolvingUtilizationOfUnsecuredLines

- Data 이상치라고 판단되는 값을 Max. value로 대체함

In [ ]:
# replace value
preprocessing_data.loc[preprocessing_data['RevolvingUtilizationOfUnsecuredLines'] > 1., 'RevolvingUtilizationOfUnsecuredLines'] = 1.

In [ ]:
# 결과 확인
preprocessing_data[preprocessing_data['RevolvingUtilizationOfUnsecuredLines'] > 1.].value_counts()

### 이상치 제거 : DebtRatio

- Data의 속성상 **Highest** 이상의 숫자는 이상치를 대체함

In [ ]:
# lowest, highest value 및 index 확인
lowest, highest, lowest_outlier_idx, highest_outlier_idx = get_outlier(preprocessing_data, 'DebtRatio')
lowest, highest, lowest_outlier_idx, highest_outlier_idx

In [ ]:
# 이상치 값을 highest로 대체
preprocessing_data.loc[highest_outlier_idx, 'DebtRatio'] = highest

In [ ]:
# 결과 확인
preprocessing_data.describe()

### 이상치 제거  : NumberOfTimes90DaysLate

- 위에서 확인한 것과 같이 96번과 98번 연체 횟수는 기간적 측면에서 볼 때 말이 되지 않는 값이라 outlier로 판단하고 데이터의 수 측면에서 크지 않다고 생각되어 삭제한다.

In [ ]:
preprocessing_data[(preprocessing_data['NumberOfTimes90DaysLate'] == 98 ) | (preprocessing_data['NumberOfTimes90DaysLate'] == 96)].describe()

In [ ]:
# 삭제할 index 추출 & 삭제
data_del_idx = preprocessing_data[(preprocessing_data['NumberOfTimes90DaysLate'] == 98) | (preprocessing_data['NumberOfTimes90DaysLate'] == 96)].index
preprocessing_data.drop(data_del_idx, inplace=True)
data_del_idx.shape, preprocessing_data.shape

In [ ]:
결과 확인
preprocessing_data['NumberOfTimes90DaysLate'].value_counts()

### Result

- 각 속성에서 이상치로 판단 되는 값을 삭제 또는 대체하는 방향으로 진행.
 추가적으로 NumberRealEstateLoansOrLines에 54건수에 대한 데이터가 이상치로 보이지만 큰 영향은 안 줄 것으로 판단하고 진행했으며, 아래와 같이 정제된 data를 통해 다시 descibe 및 correlation을 파악하는 과정을 거침.

In [ ]:
# column 별 상관 관계 분석
df_corr = preprocessing_data.corr()

plt.figure(figsize=(15, 10))
plt.title('correlation')
sns.heatmap(df_corr, 
            annot = True,      
            cmap = 'RdYlBu_r',
            fmt='.3f',
            vmin = -1, vmax = 1)

In [ ]:
preprocessing_data.to_csv('preprocessing_data_v01.csv', index=False)

### **Analysis**
각 요소들의 상관관계에 관한 heatmap은 위의 그림과 같다.

양의 상관 관계에서는 대표적으로 주택 담보대출을 포함한 부동산 담보 대출 건수(NumberRealEstateLoansOrLines)와 대출자가 보유중인 담보 대출 및 신용 대출 건수(NumberOfOpenCreditLinesAndLoans)가 **0.43** 정도의 상관 관계를 보였다.

다음으로 음의 상관 관계는 대표적으로 주택 담보대출을 포함한 부동산 담보 대출 건수 (NumberRealEstateLoansOrLines)와 나이(age)가 **-0.275** 정도를 보였음을 확인 할 수 있었다.

# 2. Base line model

## 결측치 Preprocessing 적용한 Data 활용

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline


from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_roc_curve, plot_confusion_matrix, roc_auc_score

In [ ]:
preprocessing_data = pd.read_csv('preprocessing_data_v01.csv')
preprocessing_data.head()

In [ ]:
X = preprocessing_data.drop('SeriousDlqin2yrs', axis=1)
y = preprocessing_data['SeriousDlqin2yrs']

X.shape, y.shape

In [ ]:
# train / validataion / test set split & imbalanced data processing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=157,
                                                    stratify=y
                                                    )

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                test_size=0.2,
                                                random_state=157,
                                                stratify=y_train
                                                )

smt = SMOTE(random_state=157)
X_train, y_train = smt.fit_resample(X_train, y_train)

X_train.shape, X_val.shape, X_test.shape

In [ ]:
knn = make_pipeline(scaler, KNeighborsClassifier())
lr = make_pipeline(scaler, LogisticRegression(max_iter=2000, random_state=0))
rf = RandomForestClassifier(random_state=0)
grb = GradientBoostingClassifier(random_state=0)
xgb = XGBClassifier(random_state=0)

In [ ]:
knn.fit(X_train, y_train)
lr.fit(X_train, y_train)
xgb.fit(X_train, y_train)
grb.fit(X_train, y_train)
rf.fit(X_train, y_train)

In [ ]:
base_line = [knn, lr, xgb, grb, rf]
model_names = ['KNN', 'LogisticRegression', 'XGBoost', 'GradientBoosting', 'RandomForest']

In [ ]:
for model, name in zip(base_line, model_names):
    if name == 'KNN':
        continue
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    pred_train_proba = model.predict_proba(X_train)
    pred_test_proba = model.predict_proba(X_test)
    
    acc_train = np.round(accuracy_score(y_train,pred_train),3)
    acc_test = np.round(accuracy_score(y_test, pred_test), 3)
    
    auc_train = np.round(roc_auc_score(y_train, pred_train_proba[:, 1]), 3)
    auc_test = np.round(roc_auc_score(y_test, pred_test_proba[:, 1]), 3)
    
    print(f'{name}')
    print(f'train정확도:{acc_train}, Test정확도:{acc_test}\t train AUC:{auc_train}, Test AUC:{auc_test}')
    print('='*50)

### Results
처음 데이터의 경우, 불균형 데이터임을 확인하였기에, 이를 조금 해소하고자 train data에 SMOTE를 통해서 불균형을 어느정도 해소를 했다.

GradientBoostingClassifier, RandomForestClassifier, LogisticRegression, KNeighborsClassifier, XGBClassifier 등의 model들을 활용했다. roc_auc_score는 대부분 **0.7**이상의 score를 가진 것을 확인했다.

# 3. Hyper-parameter Tuning

In [ ]:
grb = GradientBoostingClassifier(random_state=157)

param = {
    'learning_rate': [0.1, 0.5, 1],
    'n_estimators':range(500, 1001, 100),
    'max_depth': range(1, 4),
    'subsample':[0.5, 0.8, 1]
}

gs = RandomizedSearchCV(grb, 
                  param, 
                  scoring='accuracy',
                  cv=4,
                  n_iter=60,
                  n_jobs=-1)

gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_, gs.best_score_

In [ ]:
df = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')
df.head()

In [ ]:
best_grb = gs.best_estimator_
pred_train = best_grb.predict(X_train)
pred_val = best_grb.predict(X_val)

print_metrics_classifier(y_train, pred_train, 'grb train')
print_metrics_classifier(y_val, pred_val, 'grb validation')

In [ ]:
# 순서대로 pipeline 구성 : Feature scaling => lr로 학습/추론
# 각 프로세스 등록 ("이름", 객체)

order = [
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression())
]
pipeline = Pipeline(order, verbose=True) # verbose=True : 어떤 단계 처리하는지 log를 남김
print(pipeline.steps)

param = {
    'lr__C':[0.001, 0.01, 0.1, 1, 10, 100],
    'lr__max_iter': range(100, 1001, 100)
}

gs = GridSearchCV(pipeline, 
                  param, 
                  scoring='accuracy', 
                  cv=5, 
                  n_jobs=-1)

gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_, gs.best_score_

In [ ]:
df = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')
df.head()

In [ ]:
best_lr = gs.best_estimator_
pred_train = best_lr.predict(X_train)
pred_val = best_lr.predict(X_val)

print_metrics_classifier(y_train, pred_train, 'lr train')
print_metrics_classifier(y_val, pred_val, 'lr validation')

In [ ]:
rf = RandomForestClassifier()

param = {
    'n_estimators':[0.01, 0.1, 1, 10],
    'max_depth':[1, 2, 3, 4],
    'max_leaf_nodes':[5, 10, 15, 20, 30],
    'min_samples_leaf': range(500, 1001, 100)
}

gs = RandomizedSearchCV(rf, 
                param, 
                scoring='accuracy', 
                cv=4,
                n_iter=50,
                n_jobs=-1)

gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_, gs.best_score_

In [ ]:
df = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')
df.head()

In [ ]:
best_rf = gs.best_estimator_
pred_train = best_rf.predict(X_train)
pred_val = best_rf.predict(X_val)

print_metrics_classifier(y_train, pred_train, 'RandomForest train')
print_metrics_classifier(y_val, pred_val, 'RandomForest validation')

In [ ]:
estimators = [
    ('lr', best_lr),
    ('grb', best_grb),
    ('rf', best_rf)
]

voting = VotingClassifier(estimators, voting='soft')
voting.fit(X_train, y_train)

pred_train = voting.predict(X_train)
pred_val = voting.predict(X_val)

print_metrics_classifier(y_train, pred_train, 'voting train')
print_metrics_classifier(y_val, pred_val, 'voting validation')

In [ ]:
pred_test = voting.predict(X_test)
print_metrics_classifier(y_test, pred_test, 'voting test')

# 4. Result

### ROC_AUC curve

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(y_test, pred_test)
disp = ConfusionMatrixDisplay(cm, # confusion matrix
                            display_labels=['SeriousDlqin2yrs: No', 'SeriousDlqin2yrs: Yes']) # [Negative label, Positive label]
disp.plot(cmap='Reds') # 출력

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_curve

voting_pos = voting.predict_proba(X_test)[:, 1]

fprs_voting, tprs_voting, thresholds_voting = roc_curve(y_test, voting_pos)

print(fprs_voting.shape, tprs_voting.shape, thresholds_voting.shape)

auc = roc_auc_score(y_test, pred_test)
disp = RocCurveDisplay(fpr=fprs_voting, tpr=tprs_voting, roc_auc=auc)
disp.plot()
plt.legend(loc='lower right')
plt.title('ROC Curve')
plt.grid()

### Results
Base line model중에서 꽤 괜찮은 성능을 보였던 RandomForest, LogisticRegression, Gradient Boosting model을 선정했다. 이 세 가지 모델들을 통해서 GridSearchCV와 RandomGridSearchCV를 이용해 hyper-parameter tuning을 진행했다. 다시 그 best model들을 통해서 validation을 진행했으며, Voting Classifier를 통해서 모델을 완성했다.

결과적으로는 test_data와 ROC_AUC curve를 통해 살펴본 결과, **roc_auc_score**는 **0.78**로 나왔습니다. 따라서 연체한 사람들에 대해 어느정도 유의미하게 판단할 수 있음을 알 수 있었다.

최근 2년 동안 90일 이상 연체하지 않은 비율이 연체한 비율에 비해서 매우 높기 때문에, data적인 측면에서는 연체한 사람들에 대한 case가 더 많이 필요함을 알 수 있었다.

모델의 성능을 더 올리기 위해 다양한 이상치를 처리하거나 제거하는 방법들을 이용해보았지만, 현재의 요소를 통해서는 이 이상의 성능을 내는 것이 어려웠다. 그리고 요소별로 서로의 상관관계가 너무 낮은 것도 모델의 성능을 더 올리기 어려운 이유중 하나라고 생각된다. 마지막으로 Xgboosting, GradientBoosting classifier의 학습 시간이 상당히 오래 걸리는 부분도 모델의 평가를 빠르게 하지 못하게 되어 아쉬운 부분이 있었다.